# Lab 03
Authors: Santiago del Rey Juárez and Nikita Belooussov

## Session Excercise
### 1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project.



In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
dt = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IHLT/data/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [30]:
dt['gs'] = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IHLT/data/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

In [31]:
dt.shape

(459, 3)

In [32]:
dt.head()

,0,1,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


### 2. Compute their similarities by considering lemmas and Jaccard distance.

In [33]:
wnl = nltk.stem.WordNetLemmatizer()

In [34]:
from nltk.corpus import wordnet
def lemmatize(p):
  if p[1][0] in {'N','V','J','R'}: #N- noun, V- verb, J- adjective, R-adverb
    if p[1][0] is 'J':  #this is used due to wordnet using a different label for adjectives than one given by nltk
      return wnl.lemmatize(p[0].lower(), pos=wordnet.ADJ)
    return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
  return p[0]

def compute_jaccard(sentence_0, sentence_1, type = 'lemma'):
  s0 = nltk.word_tokenize(sentence_0)
  s1 = nltk.word_tokenize(sentence_1)
  if type == 'lemma':
    pairs_s0 = nltk.pos_tag(s0)
    pairs_s1 = nltk.pos_tag(s1)
    s0 = [lemmatize(pair) for pair in pairs_s0]
    s1 = [lemmatize(pair) for pair in pairs_s1]
  
  return 1 - jaccard_distance(set(s0), set(s1))

In [35]:
dt['jaccard_lemma'] = pd.Series(
    map(lambda s0, s1: compute_jaccard(s0, s1, 'lemma'), dt[0], dt[1])
)
dt.head()

,0,1,gs,jaccard_lemma
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50,0.346154
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,0.923077
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25,0.391304
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50,0.545455
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00,1.000000


### 3. Compare the results with those in session 2 (document structure) in which words were considered.

In [36]:
dt['jaccard_word'] = pd.Series(
    map(lambda s0, s1: compute_jaccard(s0, s1, 'word'), dt[0], dt[1])
)
dt.head()

,0,1,gs,jaccard_lemma,jaccard_word
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50,0.346154,0.346154
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,0.923077,0.785714
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25,0.391304,0.391304
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50,0.545455,0.545455
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00,1.000000,1.000000


In [37]:
dt.query('jaccard_lemma > jaccard_word')

,0,1,gs,jaccard_lemma,jaccard_word
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,0.923077,0.785714
8,The leaders have now been given a new chance a...,Leaders now have another chance to let them an...,5.00,0.318182,0.260870
11,We often pontificate here about being the repr...,We are proud often here to represent the citiz...,3.50,0.600000,0.500000
14,I am one of those Members who attends sittings...,I belong to the Members who are present rather...,4.25,0.277778,0.210526
17,"As I already explained during second reading, ...","As I have already explained, at second reading...",4.50,0.565217,0.500000
...,...,...,...,...,...
438,That provision could open the door wide to arb...,This regulation opens the door to arbitrariness.,4.25,0.500000,0.384615
439,The European Union has got to do something and...,It is appropriate that the European Union is i...,4.00,0.388889,0.315789
440,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,0.923077,0.785714
443,Then perhaps we could have avoided a catastrophe.,We would perhaps have been able to then avoid ...,4.25,0.400000,0.235294


In [38]:
dt.query('jaccard_lemma < jaccard_word')



,0,1,gs,jaccard_lemma,jaccard_word
19,Amendment No 7 proposes certain changes in the...,Amendment No 7 proposes changes to certain par...,5.0,0.571429,0.833333
42,Amendment No 7 proposes certain changes in the...,Amendment 7 proposes certain modifications in ...,5.0,0.642857,0.769231
72,Amendment No 7 proposes certain changes in the...,Amendment 7 proposes certain modifications in ...,5.0,0.642857,0.769231
132,Amendment No 7 proposes certain changes in the...,Amendment 7 proposes certain modifications in ...,5.0,0.642857,0.769231
197,"The fishermen are inactive, tired and disappoi...","The fishermen inactive, are tired and disappoi...",5.0,0.636364,1.000000


### 4. Compare the results with gold standard by giving the pearson correlation between them.

In [39]:
from scipy.stats import pearsonr
corr_lemma = pearsonr(dt['gs'], dt['jaccard_lemma'])[0]
corr_word = pearsonr(dt['gs'], dt['jaccard_word'])[0]
print('Pearson correlation between gs and jaccard_lemma:', corr_lemma)
print('Pearson correlation between gs and jaccard_word:', corr_word)

Pearson correlation between gs and jaccard_lemma: 0.4568471894478094
Pearson correlation between gs and jaccard_word: 0.4504977169318684


### 5. Questions (justify the answers):
  * Which is better: words or lemmas?
  * Do you think that could perform better for any pair of texts?

According to our results, both methods are approximately equivalent. The lemmas method performs slightly better overall, but not by a significant enough amount when comparing large amounts of texts. A closer inspection shows that the lemmas tends to perform better if the sentence structure does not change between the two compared sentences, while if the structure is changed then the words method will perform better. 
<br><br>A good example for the lemmas case can be seen in the first sentence in the dataset. Where the sentence structure is not changed, only that the verb tense is changed between the two sentences by adding an is. Due to the lemmatization of the sentence the changed verb proposing changed back to propose, thus matching the original sentence more closely.
<br><br>An example when to use the words method can be seen in sentence 197 in the dataset. In this sentence the sentence structure changes when the word inactive is moved to be before the verb. This causes some of the words in the sentence to be lemmatized differently, and not match the other sentence anymore. Since this does not happen when using the word method, it produces a better outcome. 


In [40]:
#testing why the lemma performs better than words
s0 = nltk.word_tokenize(dt.loc[1][0])
s1 = nltk.word_tokenize(dt.loc[1][1])
pairs_s0 = nltk.pos_tag(s0)
pairs_s1 = nltk.pos_tag(s1)
lemmas_s0 = [lemmatize(pair) for pair in pairs_s0]
lemmas_s1 = [lemmatize(pair) for pair in pairs_s1]

print(f"Sentence 0 after tokenizing:\n{s0}")
print(f"Sentence 0 after tokenizing and pos tagging:\n{pairs_s0}")
print(f"Sentence 0 after tokenizing, pos tagging and lemmatizing:\n{lemmas_s0}")
print()

print(f"Sentence 1 after tokenizing:\n{s1}")
print(f"Sentence 1 after tokenizing and pos tagging:\n{pairs_s1}")
print(f"Sentence 1 after tokenizing, pos tagging and lemmatizing:\n{lemmas_s1}")

Sentence 0 after tokenizing:
['Amendment', 'No', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.']
Sentence 0 after tokenizing and pos tagging:
[('Amendment', 'NNP'), ('No', 'NNP'), ('7', 'CD'), ('proposes', 'VBZ'), ('certain', 'JJ'), ('changes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('references', 'NNS'), ('to', 'TO'), ('paragraphs', 'VB'), ('.', '.')]
Sentence 0 after tokenizing, pos tagging and lemmatizing:
['amendment', 'no', '7', 'propose', 'certain', 'change', 'in', 'the', 'reference', 'to', 'paragraph', '.']

Sentence 1 after tokenizing:
['Amendment', 'No', '7', 'is', 'proposing', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.']
Sentence 1 after tokenizing and pos tagging:
[('Amendment', 'NNP'), ('No', 'NNP'), ('7', 'CD'), ('is', 'VBZ'), ('proposing', 'VBG'), ('certain', 'JJ'), ('changes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('references', 'NNS'), ('to', 'TO'), ('paragraphs', 'VB'), ('.', '.')]
Sentence 1 after to

In [41]:
#testing why the word preforms better on some sentences than lemma
s0 = nltk.word_tokenize(dt.loc[197][0])
s1 = nltk.word_tokenize(dt.loc[197][1])
pairs_s0 = nltk.pos_tag(s0)
pairs_s1 = nltk.pos_tag(s1)
lemmas_s0 = [lemmatize(pair) for pair in pairs_s0]
lemmas_s1 = [lemmatize(pair) for pair in pairs_s1]

print(f"Sentence 0 after tokenizing:\n{s0}")
print(f"Sentence 0 after tokenizing and pos tagging:\n{pairs_s0}")
print(f"Sentence 0 after tokenizing, pos tagging and lemmatizing:\n{lemmas_s0}")
print()

print(f"Sentence 1 after tokenizing:\n{s1}")
print(f"Sentence 1 after tokenizing and pos tagging:\n{pairs_s1}")
print(f"Sentence 1 after tokenizing, pos tagging and lemmatizing:\n{lemmas_s1}")

Sentence 0 after tokenizing:
['The', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.']
Sentence 0 after tokenizing and pos tagging:
[('The', 'DT'), ('fishermen', 'NNS'), ('are', 'VBP'), ('inactive', 'JJ'), (',', ','), ('tired', 'JJ'), ('and', 'CC'), ('disappointed', 'JJ'), ('.', '.')]
Sentence 0 after tokenizing, pos tagging and lemmatizing:
['The', 'fisherman', 'be', 'inactive', ',', 'tired', 'and', 'disappointed', '.']

Sentence 1 after tokenizing:
['The', 'fishermen', 'inactive', ',', 'are', 'tired', 'and', 'disappointed', '.']
Sentence 1 after tokenizing and pos tagging:
[('The', 'DT'), ('fishermen', 'NNS'), ('inactive', 'JJ'), (',', ','), ('are', 'VBP'), ('tired', 'VBN'), ('and', 'CC'), ('disappointed', 'VBN'), ('.', '.')]
Sentence 1 after tokenizing, pos tagging and lemmatizing:
['The', 'fisherman', 'inactive', ',', 'be', 'tire', 'and', 'disappoint', '.']
